In [27]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [122]:
n = 4

np.tile( [ [ 0, 1], [ 1, 0] ], ( 1//2, 1//2) ) 



array([], shape=(0, 0), dtype=int32)

In [44]:
np.arange(10,2,5)

array([], dtype=int32)

In [11]:
[0,3,5,6,7,8].filter(lambda x: x%2 == 0)

AttributeError: 'list' object has no attribute 'filter'

In [26]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [2]:
nhl_df = nhl_df[nhl_df['year'] == 2018]

In [3]:
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


In [4]:
nhl_df = nhl_df[~nhl_df['GP'].str.contains('Division',regex=False)]

In [5]:
nhl_df[['W','L']] = nhl_df[['W','L']].astype(int)
nhl_df['team'] = nhl_df['team'].astype(str)
cities['NHL'] = cities['NHL'].astype(str)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)

In [6]:
nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W']+nhl_df['L'])

In [7]:
cities['NHL'].replace("\[\w.*\]","",regex=True,inplace=True)

In [8]:
nhl_df['team'].replace("\*$","",regex=True,inplace=True)

In [9]:
NHL_cities = (cities['NHL'] != '') & (cities['NHL'] != '—')

In [10]:
# cities.loc[NHL_cities,'NHL'] = cities.loc[NHL_cities,'NHL']\
#     .apply(lambda x: re.findall(f"(?!\d.\s*)\w.*{x}",str(nhl_df['team']))[0]\
#          if bool(re.search(f"(?!\d.\s*)\w.*{x}",str(nhl_df['team'])))\
#               else  next((re.findall(f"(?!\d.\s*)\w.*{sub}",str(nhl_df['team'])) for sub in re.findall('[A-Z]{1}[a-z]*',x)),None)[0] )

In [11]:
cities.loc[NHL_cities,'NHL'] = cities.loc[NHL_cities,'NHL']\
    .apply(lambda x: re.findall(f"(?!\d.\s*)\w.*{x}",str(nhl_df['team']))[0]\
         if bool(re.search(f"(?!\d.\s*)\w.*{x}",str(nhl_df['team'])))\
              else  next((re.findall(f"(?!\d.\s*)\w.*{sub}",str(nhl_df['team'])) for sub in ['Devils','Ducks'] if sub in x),None)[0] )

In [12]:
nhl_metro = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]

In [13]:
nhl_c = pd.merge(nhl_df,nhl_metro,left_on='team',right_on='NHL',how='left')

In [14]:
nhl_c.loc[14:15,['Metropolitan area','Population (2016 est.)[8]','NHL']] = nhl_c.loc[12,['Metropolitan area','Population (2016 est.)[8]','team']].values

In [15]:
nhl_c.loc[26,['Metropolitan area','Population (2016 est.)[8]','NHL']] = nhl_c.loc[24,['Metropolitan area','Population (2016 est.)[8]','team']].values

In [16]:
nhl_c = nhl_c[['team','wl_ratio','Metropolitan area','Population (2016 est.)[8]']]

In [17]:
nhl_c['team']

0       Tampa Bay Lightning
1             Boston Bruins
2       Toronto Maple Leafs
3          Florida Panthers
4         Detroit Red Wings
5        Montreal Canadiens
6           Ottawa Senators
7            Buffalo Sabres
8       Washington Capitals
9       Pittsburgh Penguins
10      Philadelphia Flyers
11    Columbus Blue Jackets
12        New Jersey Devils
13      Carolina Hurricanes
14       New York Islanders
15         New York Rangers
16      Nashville Predators
17            Winnipeg Jets
18           Minnesota Wild
19       Colorado Avalanche
20          St. Louis Blues
21             Dallas Stars
22       Chicago Blackhawks
23     Vegas Golden Knights
24            Anaheim Ducks
25          San Jose Sharks
26        Los Angeles Kings
27           Calgary Flames
28          Edmonton Oilers
29        Vancouver Canucks
30          Arizona Coyotes
Name: team, dtype: object

In [18]:
nhl_grp = nhl_c.groupby('Metropolitan area')['wl_ratio','Population (2016 est.)[8]'].mean()
nhl_grp['Population (2016 est.)[8]'] = nhl_grp['Population (2016 est.)[8]'].astype(int)

In [19]:
nhl_grp['Population (2016 est.)[8]']

Metropolitan area
Boston                     4794447
Buffalo                    1132804
Calgary                    1392609
Chicago                    9512999
Columbus                   2041520
Dallas–Fort Worth          7233323
Denver                     2853077
Detroit                    4297617
Edmonton                   1321426
Las Vegas                  2155664
Los Angeles               13310447
Miami–Fort Lauderdale      6066387
Minneapolis–Saint Paul     3551036
Montreal                   4098927
Nashville                  1865298
New York City             20153634
Ottawa                     1323783
Philadelphia               6070500
Phoenix                    4661537
Pittsburgh                 2342299
Raleigh                    1302946
San Francisco Bay Area     6657982
St. Louis                  2807002
Tampa Bay Area             3032171
Toronto                    5928040
Vancouver                  2463431
Washington, D.C.           6131977
Winnipeg                    778489
Na

In [20]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df = nhl_df[~nhl_df['GP'].str.contains('Division',regex=False)]
nhl_df[['W','L']] = nhl_df[['W','L']].astype(int)
nhl_df['team'] = nhl_df['team'].astype(str)
cities['NHL'] = cities['NHL'].astype(str)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W']+nhl_df['L'])
cities['NHL'].replace("\[\w.*\]","",regex=True,inplace=True)
nhl_df['team'].replace("\*$","",regex=True,inplace=True)
NHL_cities = (cities['NHL'] != '') & (cities['NHL'] != '—')
cities.loc[NHL_cities,'NHL'] = cities.loc[NHL_cities,'NHL']\
.apply(lambda x: re.findall(f"(?!\d.\s*)\w.*{x}",str(nhl_df['team']))[0]\
        if bool(re.search(f"(?!\d.\s*)\w.*{x}",str(nhl_df['team'])))\
            else  next((re.findall(f"(?!\d.\s*)\w.*{sub}",str(nhl_df['team'])) for sub in ['Devils','Ducks'] if sub in x),None)[0] )
nhl_metro = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]
nhl_c = pd.merge(nhl_df,nhl_metro,left_on='team',right_on='NHL',how='left')
nhl_c.loc[14:15,['Metropolitan area','Population (2016 est.)[8]','NHL']] = nhl_c.loc[12,['Metropolitan area','Population (2016 est.)[8]','team']].values
nhl_c.loc[26,['Metropolitan area','Population (2016 est.)[8]','NHL']] = nhl_c.loc[24,['Metropolitan area','Population (2016 est.)[8]','team']].values
nhl_c = nhl_c[['team','wl_ratio','Metropolitan area','Population (2016 est.)[8]']]
nhl_grp = nhl_c.groupby('Metropolitan area')['wl_ratio','Population (2016 est.)[8]'].mean()
nhl_grp['Population (2016 est.)[8]'] = nhl_grp['Population (2016 est.)[8]'].astype(int)
population_by_region = nhl_grp['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
win_loss_by_region = nhl_grp['wl_ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
stats.pearsonr(population_by_region, win_loss_by_region)

(0.012486162921209923, 0.9497182859911781)

In [21]:
nba_df=pd.read_csv("assets/nba.csv")

In [22]:
nba_df = nba_df[nba_df['year'] == 2018]

In [23]:
nba_df[['W','L']] = nba_df[['W','L']].astype(int)
nba_df['team'] = nba_df['team'].astype(str)
cities['NBA'] = cities['NBA'].astype(str)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)

In [24]:
cities['NBA'].replace("\[\w.*\]","",regex=True,inplace=True)
nba_df['team'].replace("\**\s\(\d*\)$","",regex=True,inplace=True)

In [25]:
nba_df['wl_ratio'] = nba_df['W'] / (nba_df['W']+nba_df['L'])

In [26]:
NBA_cities = (cities['NBA'] != '') & (cities['NBA'] != '—')

In [27]:
cities.loc[NBA_cities,'NBA'] = cities.loc[NBA_cities,'NBA']\
.apply(lambda x: re.findall(f"(?!\d.\s*)\w.*{x}",str(nba_df['team']))[0]\
        if bool(re.search(f"(?!\d.\s*)\w.*{x}",str(nba_df['team'])))\
            else  next((re.findall(f"(?!\d.\s*)\w.*{sub}",str(nba_df['team'])) for sub in ['Nets','Clippers'] if sub in x),None)[0] )

In [28]:
nba_metro = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]

In [29]:
nba_mg = pd.merge(nba_df,nba_metro,left_on='team',right_on='NBA',how='left')

In [30]:
nba_mg.loc[10,['Metropolitan area','Population (2016 est.)[8]','NBA']] = nba_mg.loc[11,['Metropolitan area','Population (2016 est.)[8]','team']].values
nba_mg.loc[25,['Metropolitan area','Population (2016 est.)[8]','NBA']] = nba_mg.loc[24,['Metropolitan area','Population (2016 est.)[8]','team']].values

In [31]:
nba_mg = nba_mg[['team','wl_ratio','Metropolitan area','Population (2016 est.)[8]']]
nba_grp = nba_mg.groupby('Metropolitan area')['wl_ratio','Population (2016 est.)[8]'].mean()
nba_grp['Population (2016 est.)[8]'] = nba_grp['Population (2016 est.)[8]'].astype(int)
population_by_region = nba_grp['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
win_loss_by_region = nba_grp['wl_ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

In [32]:
assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

stats.pearsonr(population_by_region, win_loss_by_region)

(-0.17657160252844623, 0.3687474160446294)

In [33]:
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
mlb_df = mlb_df[mlb_df['year'] == 2018]
mlb_df[['W','L']] = mlb_df[['W','L']].astype(int)
mlb_df['team'] = mlb_df['team'].astype(str)
cities['MLB'] = cities['MLB'].astype(str)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
cities['MLB'].replace("\[\w.*\]","",regex=True,inplace=True)
mlb_df['team'].replace("\**\s\(\d*\)$","",regex=True,inplace=True)
mlb_df['wl_ratio'] = mlb_df['W'] / (mlb_df['W']+mlb_df['L'])
MLB_cities = (cities['MLB'] != '') & (cities['MLB'] != '—')

In [34]:
cities.loc[MLB_cities,'MLB'] = cities.loc[MLB_cities,'MLB']\
.apply(lambda x: re.findall(f"(?!\d.\s*)\w.*{x}",str(mlb_df['team']))[0]\
        if bool(re.search(f"(?!\d.\s*)\w.*{x}",str(mlb_df['team'])))\
            else  next((re.findall(f"(?!\d.\s*)\w.*{sub}",str(mlb_df['team']))\
                 for sub in ['Mets','White Sox','Angels','Athletics'] if sub in x),None)[0] )

In [36]:
mlb_metro = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
mlb_mg = pd.merge(mlb_df,mlb_metro,left_on='team',right_on='MLB',how='left')

In [37]:
mlb_mg.loc[1,['Metropolitan area','Population (2016 est.)[8]','MLB']] = mlb_mg.loc[18,['Metropolitan area','Population (2016 est.)[8]','team']].values
mlb_mg.loc[25,['Metropolitan area','Population (2016 est.)[8]','MLB']] = mlb_mg.loc[13,['Metropolitan area','Population (2016 est.)[8]','team']].values
mlb_mg.loc[28,['Metropolitan area','Population (2016 est.)[8]','MLB']] = mlb_mg.loc[11,['Metropolitan area','Population (2016 est.)[8]','team']].values
mlb_mg.loc[21,['Metropolitan area','Population (2016 est.)[8]','MLB']] = mlb_mg.loc[8,['Metropolitan area','Population (2016 est.)[8]','team']].values

In [38]:
mlb_mg = mlb_mg[['team','wl_ratio','Metropolitan area','Population (2016 est.)[8]']]
mlb_grp = mlb_mg.groupby('Metropolitan area')['wl_ratio','Population (2016 est.)[8]'].mean()
mlb_grp['Population (2016 est.)[8]'] = mlb_grp['Population (2016 est.)[8]'].astype(int)
population_by_region = mlb_grp['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
win_loss_by_region = mlb_grp['wl_ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

In [39]:
assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
assert len(population_by_region) == 26, "Q2: There should be 28 teams being analysed for NBA"

stats.pearsonr(population_by_region, win_loss_by_region)

(0.15027698302669307, 0.46370703378875583)

In [40]:
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nfl_df = nfl_df[nfl_df['year'] == 2018]
nfl_df = nfl_df[~nfl_df['W'].str.contains('[FC]',regex=True)]
nfl_df[['W','L']] = nfl_df[['W','L']].astype(int)
nfl_df['team'] = nfl_df['team'].astype(str)
cities['NFL'] = cities['NFL'].astype(str)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
cities['NFL'].replace("\[\w.*\]","",regex=True,inplace=True)
nfl_df['team'].replace("\**\+*","",regex=True,inplace=True)
nfl_df['wl_ratio'] = nfl_df['W'] / (nfl_df['W']+nfl_df['L'])
NFL_cities = (cities['NFL'] != '') & (~cities['NFL'].str.contains('[—]',regex=True))

In [41]:
cities.loc[NFL_cities,'NFL'] = cities.loc[NFL_cities,'NFL']\
.apply(lambda x: re.findall(f"(?!\d.\s*)\w.*{x}",str(nfl_df['team']))[0]\
        if bool(re.search(f"(?!\d.\s*)\w.*{x}",str(nfl_df['team'])))\
            else  next((re.findall(f"(?!\d.\s*)\w.*{sub}",str(nfl_df['team']))\
                 for sub in ['Jets','Chargers','Raiders'] if sub in x),None)[0] )

In [42]:
nfl_metro = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
nfl_mg = pd.merge(nfl_df,nfl_metro,left_on='team',right_on='NFL',how='left')

In [43]:
nfl_mg.loc[30,['Metropolitan area','Population (2016 est.)[8]','NFL']] = nfl_mg.loc[15,['Metropolitan area','Population (2016 est.)[8]','team']].values
nfl_mg.loc[28,['Metropolitan area','Population (2016 est.)[8]','NFL']] = nfl_mg.loc[13,['Metropolitan area','Population (2016 est.)[8]','team']].values
nfl_mg.loc[19,['Metropolitan area','Population (2016 est.)[8]','NFL']] = nfl_mg.loc[3,['Metropolitan area','Population (2016 est.)[8]','team']].values

In [44]:
nfl_mg = nfl_mg[['team','wl_ratio','Metropolitan area','Population (2016 est.)[8]']]
nfl_grp = nfl_mg.groupby('Metropolitan area')['wl_ratio','Population (2016 est.)[8]'].mean()
nfl_grp['Population (2016 est.)[8]'] = nfl_grp['Population (2016 est.)[8]'].astype(int)
population_by_region = nfl_grp['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
win_loss_by_region = nfl_grp['wl_ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

In [45]:
assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

stats.pearsonr(population_by_region, win_loss_by_region)

(0.004922112149349409, 0.9797833458363694)

In [46]:
mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [47]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

In [84]:
df = pd.merge(nba_grp['wl_ratio'], nfl_grp['wl_ratio'], on='Metropolitan area').merge(mlb_grp['wl_ratio'],on='Metropolitan area').merge(nhl_grp['wl_ratio'],on='Metropolitan area')
df.columns = ['NBA','NFL','MLB','NHL']
#next_df = stats.ttest_rel(df_nba_nhl['w_l ratio_x'], df_nba_nhl['w_l ratio_y'])

In [85]:
for col1 in df.columns:
    for col2 in df.columns:
         p_values.loc[col1,col2]  = stats.ttest_rel(df[col1],df[col2])[0]

In [88]:
stats.ttest_rel(df['MLB'],df['NFL'])

Ttest_relResult(statistic=0.29655494651389214, pvalue=0.7718764122646871)

In [89]:
import numpy as np

a = np.arange(8)
b = a[4:6]
b[:] = 40
c = a[4] + a[6]

In [90]:
c

46

In [91]:
import re
s = 'ABCAC'

In [96]:
bool(re.match('A', s)) == True

True

In [97]:
    s = 'ACAABAACAAABACDBADDDFSDDDFFSSSASDAFAAACBAAAFASD'


In [131]:
re.finditer('((?<=AAA).)',s)

In [132]:
for item in re.finditer('((?<=AAA).)', s):
      # identify the group number below.
      print(item.group())

B
C
F


In [174]:
df = pd.Series([4,5,-7,3],index=['d','b','a','c'])

In [179]:
df[0]

4

In [148]:
df = pd.DataFrame(np.arange(1,10,1).reshape(3,3))

In [149]:
df

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9


In [150]:
f = lambda x: x.max() + x.min()
df_new = df.apply(f)

In [152]:
df_new[0]

8

In [172]:
def result():
    s = 'ACAABAACAAABACDBADDDFSDDDFFSSSASDAFAAACBAAAFASD'

    result = []
    # compete the pattern below
    pattern = '((?<=AAA).)'
    for item in re.finditer(pattern, s):
      # identify the group number below.
      result.append(item.group(0))
      
    return result

In [173]:
result()

['B', 'C', 'F']

In [180]:
s1 = pd.Series([20,15,18,31],index=['M','S','B','V'])
s2 = pd.Series([20,30,15,20,20],index=['S','V','B','M','P'])

In [181]:
s3= s1.add(s2)

In [188]:
s3['P'] >= s3['M']

False

In [189]:
S = pd.Series(np.arange(5), index=['a', 'b', 'c', 'd', 'e'])

In [193]:
S[1:4]

b    1
c    2
d    3
dtype: int32